In [10]:
import pandas as pd
import pickle, math

# Unpickle the DataFrame
with open('sensor_data.pkl', 'rb') as file:
    sensor_df = pickle.load(file)

sensor_df["pv_change"] = sensor_df["PV0"] - sensor_df["PV1"]
sensor_df["T_change"] = sensor_df["Temp_Top"] - sensor_df["Temp_Bot"]

In [13]:
forecast_col = "pv_change"
sensor_df.fillna(-99999, inplace=True)

forecast_out = int(math.ceil(0.1*len(sensor_df)))

sensor_df["label"] = sensor_df[forecast_col].shift(-forecast_out)
sensor_df.dropna(inplace=True)
sensor_df.tail()

,Time,PV0,PV1,Temp_Top,Temp_Bot,pv_change,T_change,label
7073,1.695434e+09,0.583445,0.484472,20.3750,19.1875,0.098973,1.1875,0.101265
7074,1.695435e+09,0.583470,0.484477,20.3750,19.1250,0.098992,1.2500,0.101187
7075,1.695435e+09,0.583448,0.484500,20.3125,19.0625,0.098948,1.2500,0.101205
7076,1.695436e+09,0.583550,0.484491,20.2500,19.0625,0.099059,1.1875,0.101183
7077,1.695437e+09,0.583534,0.484459,20.2500,19.0000,0.099075,1.2500,0.101132
